# Relation Example

In [19]:
import os
import sys
import pprint

import pandas as pd

from tqdm import tqdm

notebook_dir = os.getcwd()

sys.path.append(os.path.join(notebook_dir, '../'))

from data_processing import DataProcessing
from prediction_properties import PredictionProperties
from text_generation_models import TextGenerationModelFactory

In [20]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_columns', 40)
# pd.set_option('display.max_rows', None)

## Load Data

In [21]:
base_data_path = os.path.join(notebook_dir, '../data')
combine_data_path = os.path.join(base_data_path, 'financial_phrase_bank/combined_generated_fin_phrase_bank')
model_results_path = os.path.join(combine_data_path, 'sentence_label-all_classifiers_with_results-v1.csv')
df = DataProcessing.load_from_file(model_results_path, 'csv', sep=',')
df.head(3)

,Base Sentence,Sentence Label,Author Type,Embedding,Normalized Embeddings,perceptron,sgd_classifier,logistic_regression,ridge_classifier,decision_tree_classifier,random_forest_classifier,gradient_boosting_classifier,llama-3.1-70b-instruct,llama-3.1-8b-instruct,llama-3.3-70b-instruct,mistral-7b-instruct,mistral-small-3.1,codestral-22b,gpt-oss-20b,gpt-oss-120b,granite-3.3-8b-instruct
0,Aspocomp intends to set up a plant to manufacture printed circuit boards with an investment of Rs310 crore .,0,1,[ 3.10125723e-02 1.48145273e-01 1.18822167e-02 -4.00886834e-02\n 2.18477882e-02 -7.52243698e-02 -6.43576309e-02 -1.18781418e-01\n -9.42035094e-02 1.69367039e+00 -1.97394520e-01 6.25598105e-03\n 4.40751575e-02 -4.00502793e-02 -4.52028252e-02 -1.13160096e-01\n 8.78665820e-02 1.32680905e+00 -4.84675802e-02 -3.56700621e-03\n 5.54845780e-02 9.94077399e-02 -1.34666905e-01 1.01711877e-01\n 6.37347773e-02 5.76260537e-02 -8.55392031e-03 -5.17340861e-02\n -9.67892632e-02 -4.00202125e-02 -4.66998853e-02 2.48341095e-02\n -1.75986104e-02 2.24689487e-02 4.00095731e-02 -1.50968343e-01\n 3.12316287e-02 8.40115175e-02 6.51817173e-02 -1.11669578e-01\n 4.81054671e-02 9.21993256e-02 1.43066615e-01 -2.96952873e-01\n -6.12954162e-02 -9.03191715e-02 3.97677906e-02 -3.97728458e-02\n -7....,[ 1.4120030e+00 -1.1126934e+00 8.9782916e-02 -6.6743813e-02\n -3.7037823e-01 -6.0530519e-01 -9.2802191e-01 -1.2312902e+00\n -1.7777984e+00 -3.0181044e-01 9.9699152e-01 -2.8869405e-01\n -4.5153311e-01 -2.6786438e-01 -5.6393570e-03 -1.2585411e+00\n 1.6161768e+00 1.3745358e+00 1.0518684e+00 4.4895628e-01\n 5.3809673e-01 7.0892632e-01 -1.8697438e+00 2.5000875e+00\n 4.8316771e-01 -7.1109168e-02 1.5739738e+00 -1.1464823e+00\n -1.6519629e+00 -1.5582638e+00 -8.2016826e-01 3.2399574e-01\n -2.7373379e-01 -8.1674218e-01 -2.3484817e-02 -2.1448696e+00\n 6.6440696e-01 6.2469262e-01 1.1777695e+00 -1.1532152e+00\n 6.7729610e-01 7.1531069e-01 8.3312702e-01 -4.0233145e+00\n -1.2297469e+00 -1.5874187e+00 1.4470621e+00 2.0973889e-02\n -1.5879422e+00 3.4062797e-01 -3.7409711e-01 -4.989...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Finnish consulting and engineering group Poyry Plc ( OMX Helsinki : POY ) said on Wednesday ( 1 October ) that it has been awarded a contract by Tanqia Dibba FZC as owner-engineer for the wastewater system of Dibba , Emirate of Fujairah , UAE .",0,1,[ 4.70041996e-03 1.10228069e-01 -4.77573602e-03 -9.56104398e-02\n 5.52534722e-02 -7.00817332e-02 -2.05142405e-02 -1.88917577e-01\n -1.04249775e-01 1.33263934e+00 -2.44876042e-01 6.75175413e-02\n -1.49942748e-02 -4.06556427e-02 -5.41962422e-02 -5.62920608e-02\n -5.34963198e-02 8.25075388e-01 -4.66980301e-02 -1.90773290e-02\n 9.21309367e-02 -4.51208055e-02 -1.37235259e-03 2.78238151e-02\n 5.95768094e-02 7.14695230e-02 -6.53408244e-02 -2.12525483e-02\n 1.03413671e-01 4.48262393e-02 4.36710976e-02 -1.38843171e-02\n 3.21916752e-02 8.28981474e-02 3.51038612e-02 6.20842911e-03\n 1.43408813e-02 3.44430916e-02 -3.06746140e-02 -1.25587946e-02\n 5.69351614e-02 2.96466853e-02 3.29996310e-02 3.74354306e-03\n -6.46679699e-02 -1.41989226e-02 -7.01283365e-02 2.36671865e-02\n 2....,[ 1.09362805e+00 -1.62451851e+00 -1.65100798e-01 -9.56259787e-01\n 4.15408053e-02 -5.33732831e-01 -1.78139552e-01 -2.06649685e+00\n -1.91614807e+00 -1.50415599e+00 4.32611406e-01 4.96168882e-01\n -1.36563015e+00 -2.76857466e-01 -9.52787399e-02 -3.11303347e-01\n -7.22661614e-01 -1.60111678e+00 1.07873595e+00 2.00796470e-01\n 1.14176524e+00 -1.31319058e+00 6.96443990e-02 1.48120904e+00\n 4.19971019e-01 1.25214577e-01 8.07624400e-01 -6.86450183e-01\n 1.40184307e+00 -3.70196342e-01 7.16911912e-01 -2.34501705e-01\n 5.22707343e-01 1.74109451e-02 -9.85290855e-02 4.73820418e-01\n 4.31317925e-01 -1.41181558e-01 -1.94443032e-01 1.55737445e-01\n 8.26632380e-01 -2.50532269e-01 -5.49674511e-01 6.59554422e-01\n -1.27693951e+00 -3.35813284e-01 -1.00587562e-01 7.66250908e-01\n -1....,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Diluted EPS rose to EUR3 .68 from EUR0 .50 .,0,1,[-2.69290894e-01 4.16

## Majority Vote (Prediction)

### Majority Vote

In [22]:
model_results_df = df.drop(columns=['Author Type', 'Embedding', 'Normalized Embeddings'])
model_results_df.head(3)

,Base Sentence,Sentence Label,perceptron,sgd_classifier,logistic_regression,ridge_classifier,decision_tree_classifier,random_forest_classifier,gradient_boosting_classifier,llama-3.1-70b-instruct,llama-3.1-8b-instruct,llama-3.3-70b-instruct,mistral-7b-instruct,mistral-small-3.1,codestral-22b,gpt-oss-20b,gpt-oss-120b,granite-3.3-8b-instruct
0,Aspocomp intends to set up a plant to manufacture printed circuit boards with an investment of Rs310 crore .,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Finnish consulting and engineering group Poyry Plc ( OMX Helsinki : POY ) said on Wednesday ( 1 October ) that it has been awarded a contract by Tanqia Dibba FZC as owner-engineer for the wastewater system of Dibba , Emirate of Fujairah , UAE .",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Diluted EPS rose to EUR3 .68 from EUR0 .50 .,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
model_results_df['Majority Vote'] = model_results_df.iloc[:, 1:].mode(axis=1)
model_results_df.head(3)

,Base Sentence,Sentence Label,perceptron,sgd_classifier,logistic_regression,ridge_classifier,decision_tree_classifier,random_forest_classifier,gradient_boosting_classifier,llama-3.1-70b-instruct,llama-3.1-8b-instruct,llama-3.3-70b-instruct,mistral-7b-instruct,mistral-small-3.1,codestral-22b,gpt-oss-20b,gpt-oss-120b,granite-3.3-8b-instruct,Majority Vote
0,Aspocomp intends to set up a plant to manufacture printed circuit boards with an investment of Rs310 crore .,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Finnish consulting and engineering group Poyry Plc ( OMX Helsinki : POY ) said on Wednesday ( 1 October ) that it has been awarded a contract by Tanqia Dibba FZC as owner-engineer for the wastewater system of Dibba , Emirate of Fujairah , UAE .",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Diluted EPS rose to EUR3 .68 from EUR0 .50 .,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Filter for Prediction (=1)

In [24]:
filt_prediction = (model_results_df['Majority Vote'] == 1)
predictions_df = model_results_df[filt_prediction]
predictions_df.shape

(150, 19)

In [25]:
predictions_df.head(7)

,Base Sentence,Sentence Label,perceptron,sgd_classifier,logistic_regression,ridge_classifier,decision_tree_classifier,random_forest_classifier,gradient_boosting_classifier,llama-3.1-70b-instruct,llama-3.1-8b-instruct,llama-3.3-70b-instruct,mistral-7b-instruct,mistral-small-3.1,codestral-22b,gpt-oss-20b,gpt-oss-120b,granite-3.3-8b-instruct,Majority Vote
3,Professor Thompson forecasts that the graduation rate at Harvard University will drop in 2027.,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1
6,"Economist Dr. Sarah Lee predicts on 12/31/2027, the consumer confidence index may rise.",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
13,"According to a fitness expert, the nutritional intake at community centers would fall in 21 August 2024.",1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,1,1,1
18,"The nutritional awareness in Europe should stay the same in 2028, according to a research report.",1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,0,1,1
19,"Coach Sofia Rodriguez predicts on 08/10/2028, the win ratio at the Boston Celtics may rise.",1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1
20,"Analyst Kevin Jackson predicts on 21 August 2024, the score average at the New England Patriots may rise.",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
24,The National Oceanic and Atmospheric Administration forecasts that the precipitation levels at New Orleans may decrease in 2024-08-21.,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


## LLM for Relation Extraction

### Prompts

In [26]:
prediction_base_prompt = DataProcessing.load_prediction_properties()
prediction_base_prompt

' A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:\n\n            1. <p_s>\n                - Defined as: \n                    - Source entity that states the <p>\n                - Characteristics:\n                    - A person with either: a name only, profile name only, geneder only, domain specific title only or any combination of these.\n                    - An associated organization\n                    - Named entity: Person, organization\n                    - Part of speech: Noun\n\n            2. <p_t>\n                - Defined as: \n                    - Target entity that the <p> is about\n                - Characteristics:\n                    - Same and <p_s>\n                    \n            3. <p_d>\n                - Defined as: \n                    - Date when the <p> is made\n                    - Date when the <p> is expected to come to fruition\n                - Characteristics:\n                    - For

In [27]:
# role_prompt = "Role: You are a lingust that specializes in identifying properties within a prediction statement."
# task_prompt = "Your task is label the prediction properties within"

In [28]:
system_identity_prompt = "You are a lingustic expert that specializes in identifying properties within a prediction statement."
# prediction_requirements = PredictionProperties.get_requirements()
task = """For each word within the sentence "label" as either a "source": 1, "target": 2, "date": 3, "outcome": 4, or "no_label": 0."""
sentence_label_format_output = """

Respond ONLY with valid JSON in this exact format: {"predicted_source_label": (1, word_from_sentence)}, {"predicted_target_label": (2, word_from_sentence)}, {"predicted_date_label": (3, word_from_sentence)}, {"predicted_outcome_label": (4, word_from_sentence)}, {"predicted_no_label": (0, word_from_sentence)}, where word_from_sentence is a list separated by a comma: [word_1_from_sentence, word_2_from_sentence, ..., word_N_from_sentence]. Each label can have same or different amoutn of N words.

Do NOT reason or provide anything other than {"predicted_source_label": (1, word_from_sentence)}, {"predicted_target_label": (2, word_from_sentence)}, {"predicted_date_label": (3, word_from_sentence)}, {"predicted_outcome_label": (4, word_from_sentence)}, {"predicted_no_label": (0, words_from_sentence)}.
"""

In [29]:
prediction_properties = PredictionProperties.get_prediction_properties()
prediction_properties_base_prompt = f"""{system_identity_prompt} For each prediction, the format is based on: 
    
    {prediction_properties}

"""
prediction_properties_base_prompt

'You are a lingustic expert that specializes in identifying properties within a prediction statement. For each prediction, the format is based on: \n\n     A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:\n\n            1. <p_s>\n                - Defined as: \n                    - Source entity that states the <p>\n                - Characteristics:\n                    - A person with either: a name only, profile name only, geneder only, domain specific title only or any combination of these.\n                    - An associated organization\n                    - Named entity: Person, organization\n                    - Part of speech: Noun\n\n            2. <p_t>\n                - Defined as: \n                    - Target entity that the <p> is about\n                - Characteristics:\n                    - Same and <p_s>\n                    \n            3. <p_d>\n                - Defined as: \n                    - Date wh

### Models

In [30]:
tgmf = TextGenerationModelFactory()

# Option 1: Specific models
# models = tgmf.create_instances(['llama-3.1-8b-instant', 'llama-3.3-70b-versatile', 'llama-3.3-70b-instruct', 'openai/gpt-oss-20b'])
models = tgmf.create_instances(['openai/gpt-oss-20b'])

# Option 2: All Groq models
# models = tgmf.create_instances(tgmf.get_groq_model_names())

# Option 3: All NaviGator models
# models = tgmf.create_instances(tgmf.get_navigator_model_names())

# Option 4: All available models
# models = tgmf.create_instances()

# Option 5: Mix and match
# custom_models = ['llama-3.1-70b-instruct', 'mistral-small-3.1', 'llama-3.1-8b-instant']
# models = tgmf.create_instances(custom_models)
models

### Prompt Models

In [31]:
import json
import re

def parse_json_response(response, reasoning=False):
    """Parse JSON response from LLM to extract label and reasoning"""
    try:
        # Extract JSON if there's extra text
        json_match = re.search(r'\{.*\}', response, re.DOTALL)
        if json_match:
            data = json.loads(json_match.group())
            if reasoning:
                return data.get('predicted_source_label'), data.get('predicted_target_label'), data.get('predicted_date_label'), data.get('predicted_outcome_label'), data.get('predicted_no_label'), data.get('reasoning')
            else:
                return data.get('predicted_source_label'), data.get('predicted_target_label'), data.get('predicted_date_label'), data.get('predicted_outcome_label'), data.get('predicted_no_label')
    except Exception as e:
        print(f"Error parsing JSON: {e}")
        if reasoning:
            return None, None
        else:
            return None  # Return single None when reasoning=False

In [32]:
def llm_certifier(sentence_to_classify: str, base_prompt: str, model, task, format_output: str):
    
      prompt = f"""{base_prompt}
      
      Sentence to extract the prediction properties: '{sentence_to_classify}'

      {task}
      
      {format_output}
      """
      idx = 1
      if idx == 1:
            print(f"\tPrompt: {prompt}")
            idx = idx + 1
      input_prompt = model.user(prompt)
      raw_text_llm_generation = model.chat_completion([input_prompt])
      # print(f"Raw response: {raw_text_llm_generation}")

      # Parse the JSON response
      label = parse_json_response(raw_text_llm_generation, reasoning=False)
      
      return raw_text_llm_generation, label

In [33]:
subset_predictions_df = predictions_df.iloc[:3, :]
subset_predictions_df

,Base Sentence,Sentence Label,perceptron,sgd_classifier,logistic_regression,ridge_classifier,decision_tree_classifier,random_forest_classifier,gradient_boosting_classifier,llama-3.1-70b-instruct,llama-3.1-8b-instruct,llama-3.3-70b-instruct,mistral-7b-instruct,mistral-small-3.1,codestral-22b,gpt-oss-20b,gpt-oss-120b,granite-3.3-8b-instruct,Majority Vote
3,Professor Thompson forecasts that the graduation rate at Harvard University will drop in 2027.,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1
6,"Economist Dr. Sarah Lee predicts on 12/31/2027, the consumer confidence index may rise.",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
13,"According to a fitness expert, the nutritional intake at community centers would fall in 21 August 2024.",1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,1,1,1


In [34]:
results = []
# len(predictions_df)
for idx, row in tqdm(subset_predictions_df.iterrows(), total=len(subset_predictions_df), desc="Processing"):
    text = row['Base Sentence']
    # print(f"{idx} --- Sentence: {text}")
    for model in models:
        # print(model.__name__())
        raw_response, llm_label = llm_certifier(text, prediction_properties_base_prompt, model, task, sentence_label_format_output)
        result = (text, raw_response, llm_label, model.__name__())
        results.append(result)

        if idx < 3:
            # print(f"{idx} --- Sentence: {text}")
            print(f"\n--- Result {idx} ---")
            pprint.pprint(result, width=120)
    
    # print()

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

	Prompt: You are a lingustic expert that specializes in identifying properties within a prediction statement. For each prediction, the format is based on: 

     A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:

            1. <p_s>
                - Defined as: 
                    - Source entity that states the <p>
                - Characteristics:
                    - A person with either: a name only, profile name only, geneder only, domain specific title only or any combination of these.
                    - An associated organization
                    - Named entity: Person, organization
                    - Part of speech: Noun

            2. <p_t>
                - Defined as: 
                    - Target entity that the <p> is about
                - Characteristics:
                    - Same and <p_s>
                    
            3. <p_d>
                - Defined as: 
                    - Date when the <p> is

Processing:  33%|███▎      | 1/3 [00:02<00:04,  2.41s/it]

Error parsing JSON: Expecting value: line 1 column 28 (char 27)
	Prompt: You are a lingustic expert that specializes in identifying properties within a prediction statement. For each prediction, the format is based on: 

     A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:

            1. <p_s>
                - Defined as: 
                    - Source entity that states the <p>
                - Characteristics:
                    - A person with either: a name only, profile name only, geneder only, domain specific title only or any combination of these.
                    - An associated organization
                    - Named entity: Person, organization
                    - Part of speech: Noun

            2. <p_t>
                - Defined as: 
                    - Target entity that the <p> is about
                - Characteristics:
                    - Same and <p_s>
                    
            3. <p_d>
         

Processing:  67%|██████▋   | 2/3 [00:10<00:05,  5.93s/it]

	Prompt: You are a lingustic expert that specializes in identifying properties within a prediction statement. For each prediction, the format is based on: 

     A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:

            1. <p_s>
                - Defined as: 
                    - Source entity that states the <p>
                - Characteristics:
                    - A person with either: a name only, profile name only, geneder only, domain specific title only or any combination of these.
                    - An associated organization
                    - Named entity: Person, organization
                    - Part of speech: Noun

            2. <p_t>
                - Defined as: 
                    - Target entity that the <p> is about
                - Characteristics:
                    - Same and <p_s>
                    
            3. <p_d>
                - Defined as: 
                    - Date when the <p> is

Processing: 100%|██████████| 3/3 [00:18<00:00,  6.26s/it]


In [35]:
results

[('Professor Thompson forecasts that the graduation rate at Harvard University will drop in 2027.',
  '{"predicted_source_label": (1, ["Professor", "Thompson"])}, {"predicted_target_label": (2, ["graduation", "rate", "at", "Harvard", "University"])}, {"predicted_date_label": (3, ["2027."])}, {"predicted_outcome_label": (4, ["drop"])}, {"predicted_no_label": (0, ["forecasts", "that", "the", "will", "in"])}',
  None,
  'openai/gpt-oss-20b'),
 ('Economist Dr. Sarah Lee predicts on 12/31/2027, the consumer confidence index may rise.',
  '{"predicted_source_label":[1,["Economist","Dr.","Sarah","Lee"]],"predicted_target_label":[2,["consumer","confidence","index"]],"predicted_date_label":[3,["12/31/2027"]],"predicted_outcome_label":[4,["may","rise"]],"predicted_no_label":[0,["predicts","on",",","the","."]] }',
  ([1, ['Economist', 'Dr.', 'Sarah', 'Lee']],
   [2, ['consumer', 'confidence', 'index']],
   [3, ['12/31/2027']],
   [4, ['may', 'rise']],
   [0, ['predicts', 'on', ',', 'the', '.']]),

In [36]:
results_df = pd.DataFrame(results)
results_df

,0,1,2,3
0,Professor Thompson forecasts that the graduation rate at Harvard University will drop in 2027.,"{""predicted_source_label"": (1, [""Professor"", ""Thompson""])}, {""predicted_target_label"": (2, [""graduation"", ""rate"", ""at"", ""Harvard"", ""University""])}, {""predicted_date_label"": (3, [""2027.""])}, {""predicted_outcome_label"": (4, [""drop""])}, {""predicted_no_label"": (0, [""forecasts"", ""that"", ""the"", ""will"", ""in""])}",None,openai/gpt-oss-20b
1,"Economist Dr. Sarah Lee predicts on 12/31/2027, the consumer confidence index may rise.","{""predicted_source_label"":[1,[""Economist"",""Dr."",""Sarah"",""Lee""]],""predicted_target_label"":[2,[""consumer"",""confidence"",""index""]],""predicted_date_label"":[3,[""12/31/2027""]],""predicted_outcome_label"":[4,[""may"",""rise""]],""predicted_no_label"":[0,[""predicts"",""on"","","",""the"","".""]] }","([1, [Economist, Dr., Sarah, Lee]], [2, [consumer, confidence, index]], [3, [12/31/2027]], [4, [may, rise]], [0, [predicts, on, ,, the, .]])",openai/gpt-oss-20b
2,"According to a fitness expert, the nutritional intake at community centers would fall in 21 August 2024.","{""predicted_source_label"":[1,[""According"",""to"",""a"",""fitness"",""expert""]],""predicted_target_label"":[2,[""the"",""nutritional"",""intake"",""at"",""community"",""centers""]],""predicted_date_label"":[3,[""21"",""August"",""2024""]],""predicted_outcome_label"":[4,[""would"",""fall""]],""predicted_no_label"":[0,[""in""]]}","([1, [According, to, a, fitness, expert]], [2, [the, nutritional, intake, at, community, centers]], [3, [21, August, 2024]], [4, [would, fall]], [0, [in]])",openai/gpt-oss-20b
